# Logistic Regression

## References

https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset

## Import Python Libraries

In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

## Import Dataset

In [1]:
# Source: https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset

# Dataset Column Overview: 
# ------------------------------------------------------------------------------

# X01: age
# X02: sex
# X03: chest pain type (4 values)
# X04: resting blood pressure
# X05: serum cholestoral in mg/dl
# X06: fasting blood sugar > 120 mg/dl
# X07: resting electrocardiographic results (values 0,1,2)
# X08: maximum heart rate achieved
# X09: exercise induced angina
# X10: oldpeak = ST depression induced by exercise relative to rest
# X11: the slope of the peak exercise ST segment
# X12: number of major vessels (0-3) colored by flourosopy
# X13: thal: 0 = normal; 1 = fixed defect; 2 = reversable defect
#   Y: target: presence of heart disease, 0 = no disease and 1 = disease.

In [14]:
# load heart.csv dataset from github
filepath1 = 'https://raw.githubusercontent.com/notfakearcher/julian/main/02_data/heart.csv'
df1 = pd.read_csv(filepath1, header = 0)
df1.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


## Data Cleaning

In [ ]:
# check data quality